In [1]:
import dspy
import anaconda_ai.integrations.litellm  # noqa: F401

In [2]:
lm = dspy.LM(
    "anaconda/Mistral-7B-Instruct-v0.3_Q4_K_M.gguf",
    cache=False,  # to demonstrate server launching
    temperature=0.1,
    optional_params={"server": {"ctx_size": 512}},
)
dspy.configure(lm=lm)

In [4]:
res = lm("what is pi?")
print(res[0])

 Pi (pronounced as "pie") is a mathematical constant that represents the ratio of a circle's circumference to its diameter. It is approximately equal to 3.14159, but its actual value is an irrational number, meaning it cannot be exactly represented as a common fraction. Pi is an important constant in mathematics, particularly in geometry and trigonometry, and it appears in various formulas related to circles and spheres.


In [5]:
fact_checking = dspy.asyncify(dspy.ChainOfThought("claims -> verdicts: list[bool]"))

res = await fact_checking(
    claims=["Python was released in 1991.", "Python is a compiled language."]
)
res

Prediction(
    reasoning=']\n1. False, Python was first released in 1991, which is later than 1991.\n2. False, Python is an interpreted language, not a compiled one.',
    verdicts=[False, False]
)

In [3]:
class QA(dspy.Signature):
    question: str = dspy.InputField()
    answer: str = dspy.OutputField()


trainset = [
    dspy.Example(question="What is 2+2?", answer="4").with_inputs("question"),
    dspy.Example(question="Capital of France?", answer="Paris").with_inputs("question"),
    dspy.Example(question="Color of grass?", answer="Green").with_inputs("question"),
]


def metric(example, prediction, trace=None):
    return example.answer.lower() in prediction.answer.lower()


predictor = dspy.Predict(QA)
optimizer = dspy.BootstrapFewShot(metric=metric, max_bootstrapped_demos=2)

optimized = optimizer.compile(predictor, trainset=trainset)

result = optimized(question="What is 3+3?")
result

  0%|                                                                                                  | 0/3 [00:00<?, ?it/s]

Output()

Output()

✓ Mistral-7B-Instruct-v0.3_q4_k_m.gguf (running)

 67%|████████████████████████████████████████████████████████████                              | 2/3 [01:12<00:36, 36.47s/it]


Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.


Prediction(
    answer=']\n6'
)

In [ ]:
embedder = dspy.Embedder(
    "anaconda/bge-small-en-v1.5/q4_k_m",
    batch_size=100,
    caching=False,  # to demonstrate server launching
)
embeddings = embedder(["hello", "world"])

In [ ]:
embeddings.shape

In [ ]:
output = await embedder.acall(["cat"])
output.shape